<div style="background-color: #007BFF; height: 4px; width: 100%;"></div>

# **All Data Cleaning Code**

In [15]:
import os
import re
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

### **Helper Functions**

In [8]:
def get_yearly_columns(base_cols, years):
    return [f"{col}{year}" for col in base_cols for year in years]

### **Data**

In [3]:
cohorts = range(2023, 2027)
years = range(2020, 2024)

In [68]:
# Read formatted linked persons
final23 = pd.read_csv("processed/final23.csv")    # Linked class of 2023
final24 = pd.read_csv("processed/final24.csv")    # Linked class of 2024
final25 = pd.read_csv("processed/final25.csv")    # Linked class of 2025
final26 = pd.read_csv("processed/final26.csv")    # Linked class of 2026
all_linked = pd.concat([final23, final24, final25, final26], ignore_index=True, axis=0)

final = {
    2023: final23,
    2024: final24,
    2025: final25,
    2026: final26,
}

<div style="background-color: #007BFF; height: 4px; width: 100%;"></div>

## **Numerical Categories**

In [ ]:
def preprocess(df):
    num_cols = df.select_dtypes(include=['int', 'float']).columns
    df[num_cols] = df[num_cols].fillna(-1)
    
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    df[cat_cols] = df[cat_cols].fillna("missing")
    
    df = df.replace({True: 1, False: 0, "missing": -1})
    
    genderid_cols = get_yearly_columns(["SBJ.FCT.GenderId", "SBJ.FCT.Sex", "SBJ.FCT.Sexuality"], years)
    df[genderid_cols] = df[genderid_cols].replace({
        "Boy/ man/ male": 1, 
        "Boy/man/male": 1,
        "Girl/ woman/ female": 2,
        "Girl/woman/female": 2,
        "Transgender male": 3,
        "Transgender boy/man/male": 3,
        "Transgender female": 4,
        "Transgender Girl/woman/female": 4,
        "Non-binary, genderqueer, or not exclusively male or female": 5,
        "Another gender": 6,
        "Not sure": 7,
        "I don't want to say": 8,
        "Male": 0,
        "Female": 1,
        "Asexual": 6,
        "Bisexual": 3,
        "Gay or Lesbian": 2,
        "I don't want to say": 10,
        "Pansexual": 5,
        "Queer": 4,
        "Something else": 7,
        "Straight or heterosexual": 1,
        "Straight": 1,
        "Unsure": 8,
        "I haven't thought about it or I don't know what this question means": 9,
        "Questioning or still figuring it out": 8,
    })
    
    ethnicity_cols = get_yearly_columns(["SBJ.FCT.Ethnicity"], years)
    df[ethnicity_cols] = df[ethnicity_cols].replace({
        "Hispanic/ Latino(a)": 1, 
        "Not Hispanic/ Latino(a)": 0,
        "Hispanic/ latino(a)": 1,
        "Not Hispanic/latino(a)": 0,
    })
    
    return df

<div style="background-color: #007BFF; height: 4px; width: 100%;"></div>

## **One-Hot Encoding**

In [114]:
def one_hot_encode(df, cat_cols):
    df[cat_cols] = df[cat_cols].applymap(float)
    encoder = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown="ignore")
    encoded_array = encoder.fit_transform(df[cat_cols])
    encoded_columns = encoder.get_feature_names_out(cat_cols)

    # Match "BaseVariableYYYY_Category"
    def reformat_column_name(col_name):
        match = re.search(r"(.*?)(\d{4})_(.*)", col_name)  
        if match:
            base_var, year, category = match.groups()
            return f"{base_var}_{category}{year}"
        return col_name

    reformatted_columns = [reformat_column_name(col) for col in encoded_columns]
    encoded_df = pd.DataFrame(encoded_array, columns=reformatted_columns)

    df_encoded = df.drop(columns=cat_cols).reset_index(drop=True)
    df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

    return df_encoded

In [101]:
ordinal_cols = {
    "INV.DBL.APSS.Q1.MindReading",
    "INV.DBL.APSS.Q2.TVRadio",
    "INV.DBL.APSS.Q3.Spying",
    "INV.DBL.APSS.Q4.Auditory",
    "INV.DBL.APSS.Q5.Controlled",
    "INV.DBL.APSS.Q6.Visual",
    "INV.DBL.APSS.Q7.Grandiosity",
    "INV.INT.ERS.Q01.Persistence1",
    "INV.INT.ERS.Q02.Sensitivity1",
    "INV.INT.ERS.Q03.IntensityArousal1",
    "INV.INT.ERS.Q04.IntensityArousal2",
    "INV.INT.ERS.Q05.Sensitivity2",
    "INV.INT.ERS.Q06.IntensityArousal3",
    "INV.INT.ERS.Q07.Sensitivity3",
    "INV.INT.ERS.Q08.Persistence2",
    "INV.INT.ERS.Q09.Sensitivity4",
    "INV.INT.ERS.Q10.Persistence3",
    "INV.INT.ERS.Q11.Persistence4",
    "INV.INT.ERS.Q12.Sensitivity5",
    "INV.INT.ERS.Q13.Sensitivity6",
    "INV.INT.ERS.Q14.Sensitivity7",
    "INV.INT.ERS.Q15.Sensitivity8",
    "INV.INT.ERS.Q16.Sensitivity9",
    "INV.INT.ERS.Q17.IntensityArousal4",
    "INV.INT.ERS.Q18.Sensitivity10",
    "INV.INT.ERS.Q19.IntensityArousal5",
    "INV.INT.ERS.Q20.IntensityArousal6",
    "INV.INT.ERS.Q21.IntensityArousal7",
    "INV.INT.PHQ4.Q1.Anxious",
    "INV.INT.PHQ4.Q2.Worried",
    "INV.INT.PHQ4.Q3.Depressed",
    "INV.INT.PHQ4.Q4.Anhedonia",
    "INV.INT.SUB.Alcohol.Past30",
    "INV.INT.SUB.Cannabis.Past30",
    "INV.INT.SUB.Cigarettes.Past30",
    "INV.INT.SUB.Cigars.Past30",
    "INV.INT.SUB.Smokeless.Past30",
    "INV.INT.SUB.Vapes.Past30",
}

quant_cols = {
    "INV.DBL.APSS.Total",
    "INV.INT.ERS.IntensityArousalTotal",
    "INV.INT.ERS.PersistenceTotal",
    "INV.INT.ERS.SensitivityTotal",
    "INV.INT.ERS.Total",
    "INV.INT.PHQ4.Anxiety",
    "INV.INT.PHQ4.Depression",
    "INV.INT.PHQ4.Total",
}

cat_cols = {
    "INV.LGL.PHQ4.Anxiety",
    "INV.LGL.PHQ4.Depression",
    "INV.INT.SI.Attempt",
    "INV.INT.SI.How",
    "INV.INT.SI.Selfharm",
    "INV.INT.SI.Thoughts",
    "INV.LGL.SUB.Alcohol.Life",
    "INV.LGL.SUB.Cannabis.Life",
    "INV.LGL.SUB.Cigarettes.Life",
    "INV.LGL.SUB.Cigars.Life",
    "INV.LGL.SUB.Smokeless.Life",
    "INV.LGL.SUB.Vapes.Life",
    "INV.LGL.HelpSeeking0",
    "INV.LGL.HelpSeeking1",
    "INV.LGL.HelpSeeking2",
    "INV.LGL.HelpSeeking3",
    "INV.LGL.HelpSeeking4",
    "INV.LGL.HelpSeeking5",
    "INV.LGL.HelpSeeking6",
    "INV.LGL.HelpSeeking7",
    "INV.LGL.HelpSeeking8",
    "INV.LGL.HelpSeeking9",
    "INV.LGL.HelpSeeking10",
    "INV.LGL.HelpSeeking11",
    "INV.LGL.HelpSeeking12",
    "INV.LGL.HelpSeeking13",
    "SBJ.FCT.Ethnicity",
    "SBJ.FCT.GenderId",
    "SBJ.FCT.Sex",
    "SBJ.FCT.Sexuality",
    "SBJ.LGL.Race.AmericanIndianAlaskaNative",
    "SBJ.LGL.Race.Asian",
    "SBJ.LGL.Race.HaitianBlackAfricanAmerican",
    "SBJ.LGL.Race.HawaiianPacificIslander",
    "SBJ.LGL.Race.MiddleEasternNorthAfrican",
    "SBJ.LGL.Race.White",
    "SBJ.LGL.Race.Multiple",
}

In [115]:
encoded23 = one_hot_encode(final23, get_yearly_columns(cat_cols, range(2020, 2024)))
encoded24 = one_hot_encode(final24, get_yearly_columns(cat_cols, range(2020, 2024)))
encoded25 = one_hot_encode(final25, get_yearly_columns(cat_cols, range(2020, 2024)))
encoded26 = one_hot_encode(final26, get_yearly_columns(cat_cols, range(2020, 2024)))

In [109]:
display(encoded23)

,Unnamed: 02020,Unnamed: 02021,Unnamed: 02022,Unnamed: 02023,IDX.INT.Origin.Database2020,IDX.INT.Origin.Database2021,IDX.INT.Origin.Database2022,IDX.INT.Origin.Database2023,IDX.INT.Origin.Record2020,IDX.INT.Origin.Record2021,...,SBJ.LGL.Race.HawaiianPacificIslander2022,SBJ.LGL.Race.HawaiianPacificIslander2023,SBJ.LGL.Race.HaitianBlackAfricanAmerican2020,SBJ.LGL.Race.HaitianBlackAfricanAmerican2021,SBJ.LGL.Race.HaitianBlackAfricanAmerican2022,SBJ.LGL.Race.HaitianBlackAfricanAmerican2023,SBJ.FCT.Ethnicity2020,SBJ.FCT.Ethnicity2021,SBJ.FCT.Ethnicity2022,SBJ.FCT.Ethnicity2023
0,1.0,722.0,2322.0,6676.0,18297.0,18297.0,36844.0,44184.0,986.0,2746.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,527.0,-1.0,-1.0,18297.0,18297.0,-1.0,-1.0,987.0,2081.0,...,-1.0,-1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0
2,3.0,-1.0,-1.0,-1.0,18297.0,-1.0,-1.0,-1.0,988.0,-1.0,...,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0
3,4.0,649.0,2289.0,6708.0,18297.0,18297.0,36844.0,44184.0,991.0,2483.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,612.0,2160.0,-1.0,18297.0,18297.0,36844.0,-1.0,994.0,2362.0,...,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8272,-1.0,-1.0,-1.0,9852.0,-1.0,-1.0,-1.0,44551.0,-1.0,-1.0,...,-1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0
8273,-1.0,-1.0,-1.0,9853.0,-1.0,-1.0,-1.0,44551.0,-1.0,-1.0,...,-1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0
8274,-1.0,-1.0,-1.0,9854.0,-1.0,-1.0,-1.0,44551.0,-1.0,-1.0,...,-1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0
8275,-1.0,-1.0,-1.0,9855.0,-1.0,-1.0,-1.0,44551.0,-1.0,-1.0,...,-1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0


#### **Standardizing the Columns in Each Year**
In some of the years, there are no responses of a certain category for a 
question, so there are different sets of columns for each year. This function
will create a new column of 0's for each category that is not recorded as a 
response for that year.

In [106]:
def standardize_years(df, years):
    for col in df.columns:
        match = re.search(r"(.+)(\d{4})", col)
        if match:
            var, year = match.groups()
            for y in years:
                var_col = f"{var}{y}"
                if var_col not in df.columns:
                    df[var_col] = 0
                    df[var_col] = df[var_col].astype('int64')
    return df    

In [107]:
encoded23 = standardize_years(encoded23, years)
encoded24 = standardize_years(encoded24, years)
encoded25 = standardize_years(encoded25, years)
encoded26 = standardize_years(encoded26, years)

In [108]:
print(encoded23["SBJ.FCT.Sexuality_5.02022"].dtype)

KeyError: 'SBJ.FCT.Sexuality_5.02022'

In [92]:
# Create the directory if it doesn't exist
output_dir = "encoded"
encoded23.to_csv(os.path.join(output_dir, "encoded23.csv"), index=False)
encoded24.to_csv(os.path.join(output_dir, "encoded24.csv"), index=False)
encoded25.to_csv(os.path.join(output_dir, "encoded25.csv"), index=False)
encoded26.to_csv(os.path.join(output_dir, "encoded26.csv"), index=False)

#### **Helper Function**

To make it easier for us to grab all category columns for a categorical variable in our one-hot encoded dataframe, we create this helper function which uses RegEx to grab those for us.

In [45]:
def get_encoded_columns(df, variables):
    cols = []
    for var in variables:
        cs = df.filter(regex=fr"^{var}.+").columns.tolist()
        unique_cs = set()
        for c in cs:
            match = re.search(r"(.+)(\d{4})", c)
            if match:
                c_name, _ = match.groups()
                unique_cs.add(c_name)
        cols += list(unique_cs)
    return cols

End of notebook :)

<div style="background-color: #007BFF; height: 4px; width: 100%;"></div>